In [2]:
import tempfile
from pathlib import Path

import pandas as pd
import py7zr

In [3]:
# Read the Parquet file

def read_parquet_gzip_7z(filepath: str | Path) -> pd.DataFrame:
    """Read a .parquet.gzip.7z file into a DataFrame.

    Extracts the 7z archive to a temp directory, then reads the inner
    parquet file with pandas. The '.gzip' in the filename refers to
    the parquet-internal compression codec, not external gzip wrapping.
    """
    with tempfile.TemporaryDirectory() as tmpdir:
        with py7zr.SevenZipFile(filepath, mode="r") as archive:
            archive.extractall(path=tmpdir)
        extracted = list(Path(tmpdir).iterdir())
        if not extracted:
            raise ValueError(f"No files found inside {filepath}")
        return pd.read_parquet(extracted[0])


In [4]:
path = ('../data/5FT0491_2026-01-27.parquet.gzip.7z')
df = read_parquet_gzip_7z(path)

In [5]:
df.columns.tolist()

['Timestamp',
 'EEC1_Engine.ActualEngPercentTorque',
 'EEC1_Engine.DriversDemandEngPercentTorque',
 'EEC1_Engine.EngSpeed',
 'EEC1_Engine.SrcAddrssOfCntrllngDvcForEngCtrl',
 'LFE_Engine.EngFuelRate',
 'EC1_Engine.EngReferenceTorque',
 'EEC2_Engine.AccelPedalPos1',
 'EEC2_Engine.ActlMaxAvailableEngPercentTorque',
 'CCVS1_Engine.BrakeSwitch',
 'CCVS1_Engine.CruiseCtrlActive',
 'CCVS1_Engine.CruiseCtrlSetSpeed',
 'CCVS1_Engine.CruiseCtrlStates',
 'CCVS1_Engine.WheelBasedVehicleSpeed',
 'DPFC1_Engine.DieselParticulateFilterLampCmd',
 'DPFC1_Engine.DieselParticulateFilterStatus',
 'DPFC1_Engine.DslPrtcltFltrActvRgnrationStatus',
 'DPFC1_Engine.DslPrtcltFltrPssvRgnrationStatus',
 'DPFC1_Engine.ExhaustSystemHighTempLampCmd',
 'AT1S1_Engine.Aftrtrtmnt1DslPrtcltFltrStLdRgnr',
 'AT1S1_Engine.DslPrtcltFltr1TmSncLstActvRgnrtn',
 'AT1S1_Engine.DslPrtclteFilter1SootLoadPercent',
 'ERC1_Retarder.ActualRetarderPercentTorque',
 'ERC1_Retarder.SrcAddrssOfCntrllngDvcFrRtrdrCtr',
 'RC_Retarder.ReferenceRe

# Columns Group By Functions, According to Gemini

## Human Inputs

- `EEC2_Engine.AccelPedalPos1`: The gas pedal position (0-100%). Key for Accel Overrides.

- `EBC1_Brakes.BrakePedalPos`: The brake pedal position. Key for Brake Overrides.

- `CCVS1_Cab_Controller.CruiseCtrl...`: Buttons the driver pressed (Resume, Set, Cancel).

- `Tablet.UserButtonPress`: Likely a custom event tagger (e.g., driver flagging a bug).

## ADAS Planning AKA Behaviors from System

- `ACC1_Radar.AdaptiveCruiseCtrlSetSpeed`: The speed the system was trying to hold.

- `ACC1_Radar.DistanceToForwardVehicle`: The gap it was trying to maintain.

- `ACC1_Radar.ForwardCollisionWarning`: Did the system think a crash was imminent?

- `iQEH3.iQTargSpeed`: The internal target speed calculated by the "intelligent" controller.

- `PCC1.PCCState / ACC1_Radar.AdaptiveCruiseCtrlMode`: The current health/state of the ADAS (Active, Standby, Faulted).

## Perception & Radar

- `ACC1_Radar.TargetDetected`: Boolean (True/False) if it sees a car.

- `ACC1_Radar.SpeedOfForwardVehicle`: Relative speed of the lead car.

- `Radar.Rx_Out_X, Radar.Ry_Out_X`: Raw Radar Targets. Rx is longitudinal distance, Ry is lateral position.

Note: You have _0 through _7. These are individual objects. If Rx_Out_0 suddenly jumps to 5 meters (ghost object) and the car brakes, you found your root cause.

- `IQCU_FCR.RadarMisalign...`: Diagnostic values for whether the radar is tilted (causes "Yo-yoing").

## Localization & Hills

- `IQVS1.iQRoadGrade / VDS_FC.Altitude`: The slope of the road. Key for "Slow approach to hills."

- `IQCU_IMU_Status_4.PathCurvature`: How sharp the curve is. Key for "Too fast in curves."

- `EH_not_localized`: Indicates if the map data (Electronic Horizon) is lost.

- `IQEH1.iQLCASpeedLimCur`: The speed limit the map thinks is active.

## Vehicle Dynamics (Physics)

- `EEC1_Engine.ActualEngPercentTorque`: How hard the engine was working.

- `EBC2_Brakes.FrontAxleSpeed`: Vehicle speed.

- `ACCS_RP.LongitudinalAccelerationExRange`: Actual G-force (did the car jerk?).

In [6]:
df.head()

,Timestamp,EEC1_Engine.ActualEngPercentTorque,EEC1_Engine.DriversDemandEngPercentTorque,EEC1_Engine.EngSpeed,EEC1_Engine.SrcAddrssOfCntrllngDvcForEngCtrl,LFE_Engine.EngFuelRate,EC1_Engine.EngReferenceTorque,EEC2_Engine.AccelPedalPos1,EEC2_Engine.ActlMaxAvailableEngPercentTorque,CCVS1_Engine.BrakeSwitch,...,TTC0017_FOC,NavSatNum,JammingSt,cwSuppress,HDOP,GpsReset,AutoMountState,file_sequence,ignition_counter,LoggerOnly
0,2026-01-27 14:57:46.000,0.0,0.0,0.0,0.0,0.0,2354.0,0.0,0.0,0.0,...,0.0,11.0,0.0,48.0,1.32,0.0,0.0,1,406,0
1,2026-01-27 14:57:46.100,0.0,0.0,0.0,0.0,0.0,2354.0,0.0,0.0,0.0,...,0.0,11.0,0.0,48.0,1.32,0.0,0.0,1,406,0
2,2026-01-27 14:57:46.200,0.0,0.0,0.0,0.0,0.0,2354.0,0.0,0.0,0.0,...,0.0,11.0,0.0,48.0,1.32,0.0,0.0,1,406,0
3,2026-01-27 14:57:46.300,0.0,0.0,0.0,0.0,0.0,2354.0,0.0,0.0,0.0,...,0.0,11.0,0.0,48.0,1.32,0.0,0.0,1,406,0
4,2026-01-27 14:57:46.401,0.0,0.0,0.0,0.0,0.0,2354.0,0.0,0.0,0.0,...,0.0,11.0,0.0,48.0,1.32,0.0,0.0,1,406,0


In [7]:
import numpy as np
import pandas as pd
import seaborn as sns

#### Override Events
1. Brake override
2. Throttle override
3. Cruise disengage override
4. Speed governer / iQC override
5. PCC system override

In [ ]:
overrides = pd.DataFrame({
    'override_id',
    'override_timestamp',
    'override_type',
    'cruise_active_pre',
    'vehicle_speed',
    'driver_input_type',
    'system_override_flag'
}, index=[0])